<a href="https://colab.research.google.com/github/mouleshgs/ml/blob/main/movie_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


In [2]:
df = pd.read_csv('/home/tmdb_5000_movies.csv')

In [4]:
df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [6]:
df = df[["original_title", "overview", "popularity"]]

In [7]:
df.head()

,original_title,overview,popularity
0,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577
1,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615
2,Spectre,A cryptic message from Bond’s past sends him o...,107.376788
3,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950
4,John Carter,"John Carter is a war-weary, former military ca...",43.926995


In [20]:
df = df.dropna()

In [21]:
df.isna().sum()


,0
original_title,0
overview,0
popularity,0


In [28]:
def top_popular_movies(n=5):
  return df.sort_values(by="popularity", ascending=False).head(n)

In [29]:
top_popular_movies()

,original_title,overview,popularity
546,Minions,"Minions Stuart, Kevin and Bob are recruited by...",875.581305
95,Interstellar,Interstellar chronicles the adventures of a gr...,724.247784
788,Deadpool,Deadpool tells the origin story of former Spec...,514.569956
94,Guardians of the Galaxy,"Light years from Earth, 26 years after being a...",481.098624
127,Mad Max: Fury Road,An apocalyptic story set in the furthest reach...,434.278564


In [30]:
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(df['overview'])

tfidf_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 125840 stored elements and shape (4800, 20978)>

In [32]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)


array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.02160368, 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.01488031, 0.        ,
        0.        ],
       ...,
       [0.        , 0.02160368, 0.01488031, ..., 1.        , 0.01608882,
        0.00701802],
       [0.        , 0.        , 0.        , ..., 0.01608882, 1.        ,
        0.01171476],
       [0.        , 0.        , 0.        , ..., 0.00701802, 0.01171476,
        1.        ]])

In [42]:
indices = pd.Series(df.index, index=df['original_title'])
indices

,0
original_title,
Avatar,0
Pirates of the Caribbean: At World's End,1
Spectre,2
The Dark Knight Rises,3
John Carter,4
...,...
El Mariachi,4798
Newlyweds,4799
"Signed, Sealed, Delivered",4800


In [61]:
def recommend_by_content(title, n=5):

    if title.capitalize() not in indices.index:
      return "movie not found"

    idx = indices[title.capitalize()]

    sim_scores = list(enumerate(cosine_sim[idx]))  # Similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)  # Sort
    sim_scores = sim_scores[1:n+1]  # Skip itself
    movie_indices = [i[0] for i in sim_scores]
    return df['original_title'].iloc[movie_indices]


In [67]:
recommend_by_content('inception')

,original_title
2897,Cypher
134,Mission: Impossible - Rogue Nation
1930,Stone Cold
914,Central Intelligence
1683,Pitch Perfect 2
